In [1]:
import autograd.numpy as np
from autograd import grad, elementwise_grad
import autograd.numpy.random as npr
from autograd.misc.optimizers import adam

In [2]:
def init_random_params(scale, layer_sizes, rs=npr.RandomState(0)):
    """Build a list of (weights, biases) tuples, one for each layer."""
    return [(rs.randn(insize, outsize) * scale,   # weight matrix
             rs.randn(outsize) * scale)           # bias vector
            for insize, outsize in zip(layer_sizes[:-1], layer_sizes[1:])]


def swish(x):
    "see https://arxiv.org/pdf/1710.05941.pdf"
    return x / (1.0 + np.exp(-x))


def f(params, inputs):
    "Neural network functions"
    for W, b in params:
        outputs = np.dot(inputs, W) + b
        inputs = swish(outputs)    
    return outputs


In [3]:
# Here is our initial guess of params:
params = init_random_params(0.1, layer_sizes=[1, 8, 1])

# Derivatives
fp = elementwise_grad(f, 1)
fpp = elementwise_grad(fp, 1)
fppp = elementwise_grad(fpp, 1)

eta = np.linspace(0, 6).reshape((-1, 1))

# This is the function we seek to minimize
def objective(params, step):
    # These should all be zero at the solution
    # f''' + 0.5 f'' f = 0
    zeq = fppp(params, eta) + 0.5 * f(params, eta) * fpp(params, eta) 
    bc0 = f(params, 0.0)  # equal to zero at solution
    bc1 = fp(params, 0.0)  # equal to zero at solution
    bc2 = fp(params, 6.0) - 1.0 # this is the one at "infinity"
    return np.mean(zeq**2) + bc0**2 + bc1**2 + bc2**2

def callback(params, step, g):
    if step % 1000 == 0:
        print("Iteration {0:3d} objective {1}".format(step,
                                                      objective(params, step)))

params = adam(grad(objective), params,
              step_size=0.001, num_iters=10000, callback=callback) 

print('f(0) = {}'.format(f(params, 0.0)))
print('fp(0) = {}'.format(fp(params, 0.0)))
print('fp(6) = {}'.format(fp(params, 6.0)))
print('fpp(0) = {}'.format(fpp(params, 0.0)))

import matplotlib.pyplot as plt
plt.plot(eta, f(params, eta))
plt.xlabel('$\eta$')
plt.ylabel('$f(\eta)$')
plt.xlim([0, 6])
plt.ylim([0, 4.5])

Iteration   0 objective [[1.11472535]]
Iteration 1000 objective [[0.00049768]]
Iteration 2000 objective [[0.0004579]]
Iteration 3000 objective [[0.00041697]]
Iteration 4000 objective [[0.00037408]]
Iteration 5000 objective [[0.00033705]]
Iteration 6000 objective [[0.00031016]]
Iteration 7000 objective [[0.00029197]]
Iteration 8000 objective [[0.00027585]]
Iteration 9000 objective [[0.00024616]]
f(0) = [[-0.00014613]]
fp(0) = 0.0003518041251638071
fp(6) = 0.999518061473252
fpp(0) = 0.32633705037026633


(0, 4.5)

In [5]:
%matplotlib inline
plt.show()

In [ ]:
import scipy.special.factorial as fact
k = 3
df = [f] # list for storing derivatives of f
for i in range(2*k):
    df.append(elementwise_grad(df[i], 1))

diffeq = (F/2)*np.sin(theta) + (A/2)*(k-m*w**2)
bc = 0

for i in range(k):
    diffeq += np.power(A, 2*k+1)*df[2*k+1]/(2**(2*k+1)*fact(k)*fact(k+1))

objective = np.mean(diffeq)**2
for i in range(2*k):
    objective += df[i](z[-1]) # need to flip z so that the largest z corresponds with z[-1] not z[0]